# Стохастический анализ (2024)
# Домашнее Задание 3

ВАШЕ ИМЯ ЗДЕСЬ

Оценка: ?? из 100


Задачи: ?? из 50

Ноутбук: ?? из 50


Дедлайн: ??, 2359МСК

Решённый ноутбук нужно засабмитить в класрум курса

Пожалуйста, называйте файл(ы) в формате <Имя>HW1.pdf или <Имя>HW1.ipynb. Например, KaledinHW1.pdf и KaledinHW1.ipynb.

In [ ]:
from tqdm import tqdm
import numpy as np

import cv2# для последнего задания, устанавливается pip install opencv-python
from skimage.draw import line as skline#pip install skimage

import matplotlib.pyplot as plt
import scipy.interpolate as spint
%matplotlib inline

В этом домашнем задании мы поисследуем численные методы для стохастических дифференциальных уравнений и познакомимся с одним интересным и далеко обещающим примером из статистической физики: диффузией Ланжевена (Langevin diffusion). Нам пригодятся солверы с семинаров, также мы допишем ещё несколько новых:  
- метод Мильштейна и 
- метод Рунге-Кутты.

Класс SDESolver и дочерние солверы нужно будет модернизировать для экспериментов, как это сделать, смотрите ниже. Пока мы оставляем обычный режим решения задачи Коши.

In [ ]:
class SDESolver:
    
    def __init__(self, b, sigma):
        self.b = b 
        self.sigma = sigma
    
    def step(self, x, t, tnext):
        """One integration step

        Raises:
            NotImplementedError: if not implemented
        """        
        raise NotImplementedError
    
    #added returnLast to not store everything
    #added kwargs to send additional arguments to step through solve
    def solve(self, x0, ts,tIds=None, returnLast=False, **kwargs):
        """Computes the solution on the given grid ts with x0 as initial condition

        Args:
            x0 float[]: initial condition (N,d,) or (d,)
            ts float[]: time grid (T,)
            tIds float[]: optional, ids in the dense time grid (sent through kwargs, needed for experiments)
        Returns:
            xs float[]: approximated solution (N,d,T) at points ts
        """
        try:
            if(len(x0.shape)==1):
                x0 = x0[None, :]
        except:
            #a number?
            x0 = np.ones([1,1])*x0
            
        if(tIds is None):
            usual=True
        else:
            usual=False
        
        if(returnLast):
            xs = x0    
        else:            
            xs = np.zeros([x0.shape[0], x0.shape[1], len(ts)])
            xs[...,0] = x0
        
        for i in np.arange(1,len(ts)):
            if(usual):
                #proceed as usual
                if(returnLast):#if no history needed, just the last point
                    xs = self.step(xs,ts[i-1],ts[i])
                else:
                    xs[...,i] = self.step(xs[...,i-1],ts[i-1],ts[i])
            else:
                #use pre-computed wiener process values (passed with kwargs) and time grid indices tIds
                if(returnLast):#if no history needed, just the last point
                    xs = self.step(xs,ts[i-1],ts[i], tId=tIds[i-1],tNextId=tIds[i], **kwargs)
                else:
                    xs[...,i] = self.step(xs[...,i-1],ts[i-1],ts[i], tId=tIds[i-1],tNextId=tIds[i], **kwargs)
        
        return xs
    
class EulerSolver(SDESolver):
    
    def __init__(self,b,sigma):
        super(EulerSolver, self).__init__(b,sigma)
        
    def step(self, x, t, tnext,tId=None,tNextId=None, ws=None):
        """Makes Euler step

        Args:
            x float[]: previous point (N,d)
            t float: previous time
            tnext float: next time
            tId: slice id to choose noise vector
            ws: given noise matrix (N,d,T)
        """            
        #b (N,d)  x (N,d)   sigma (N,d,d)
        #NEW!! improved to handle sigma as matrix
        if(x.ndim==1): #in case we still got (d,)
            x = x[None,:]
        
        if(ws is None):
            dw=np.random.standard_normal(size=x.shape)[:,None,:]*np.sqrt(tnext-t)#usual regime
        else:
            #YOUR CODE
            #dw=#else experimental
            pass
        res = x + self.b(t,x)*(tnext-t) + np.sum(self.sigma(t,x)*dw,axis=-1)
        return res

## Исследуем точность численных схем (25 баллов)

В отличии от детерминированного случая, есть несколько видов сходимостей.

**Сильный порядок сходимости** равен $k$, если для численного решения $Y_n^{\Delta t}$ верно с какой-то константой $C$   

$$
\mathbb{E} \left\vert Y_n^{\Delta t} - X_T \right\vert \leq С (\Delta t)^k.
$$

Это по сути порядок сходимости в смысле $L^p$.

**Слабый порядок сходимости** по отношению к классу функций $f \in \mathcal{C}$ равен $k$, если для численного решения $Y_n^{\Delta t}$ верно с какой-то константой $C$  
$$
\left\vert \mathbb{E}  f(Y_n^{\Delta t}) - f(X_T) \right\vert \leq С (\Delta t)^k.
$$

### Как соотносятся сильный и слабый порядок? (5 баллов)

Пусть $\mathcal{C}$ -- множество липшицевых функций с общей константной Липшица $K$. Докажите, что в этом случае слабый порядок больше или равен сильного.

ВАШЕ РЕШЕНИЕ ТУТ

### Новые численные схемы (20 баллов)

Для краткости как обычно будем писать для приращений

$$
\Delta W_k = W_{t_{k+1}} - W_{t_{k}}, \quad \Delta_k = t_{k+1} - t_k,
$$
а вместо объёмной $X^{\Delta}_{t_k}$ будем писать просто $X_{t_k}$ для численного решения и $X_t$ для истинного.

#### Метод Мильштейна

Подходит для уравнений вида

$$
dX_t = b(X_t) dt + \sigma(X_t) dW_t,
$$

где $b$ дифференцируема. За счёт этого требования можно скорректировать вычисление следующего шага:

$$
X_{t_{k+1}} = X_{t_k} + b(X_{t_k}) \Delta_k + \sigma(X_{t_k}) \Delta W_k + \frac{1}{2}\sigma(X_{t_k})\sigma'(X_{t_k})\left( (\Delta W_k)^2 - \Delta_k\right).
$$
Есть [аналог](https://etd.ohiolink.edu/acprod/odb_etd/ws/send_file/send?accession=case1597879378514956&disposition=inline) для многомерного случая, но мы не будем об этом.

In [ ]:
class MilsteinSolver(SDESolver):
    
    def __init__(self,b,sigma, sigmaDerivative):
        super(MilsteinSolver, self).__init__(b,sigma)
        self.sigmaDerivative = sigmaDerivative
        
    def step(self, x, t, tnext,tId=None, tNextId=None, ws = None):
        """Makes Milstein step (1dim method!!)

        Args:
            x float[]: previous point (N,d)
            t float: previous time
            tnext float: next time
            tId: slice id to choose noise vector
            ws: given noise matrix (N,d,T)
            
        Returns:
            float[]  (N,1)
        """            
        #b (N,d)  x (N,d)   sigma (N,d,d)
        #raise NotImplementedError
        #just for the same notation, write sigma and b as functions of (t,x)
        if(ws is None):
            #YOUR CODE
            dw = #wiener process delta W    #usual regime
        else:
            #YOUR CODE
            dw = #wiener process delta W    #else experimental
        #YOUR CODE
        res = None#
        
        return res[:,None]

### Метод Рунге-Кутты

Один из методов Рунге-Кутты. Это одношаговый, но двухэтапный метод, который выглядит так:

$$
\begin{align*}
&K_1 = X_{t_k} + \Delta b(X_{t_k}) + \sigma(X_{t_k})\sqrt{\Delta},\\
&X_{t_{k+1}} = X_{t_k} + \Delta b(X_{t_k}) + \sigma(X_{t_k})\Delta W_k + \frac{1}{2 \sqrt\Delta}\left( \sigma(K_1) - \sigma(X_{t_k})\right) \left( (\Delta W_k)^2 - \Delta\right),
\end{align*}
$$
Разрабатывать схемы более высокой точности в стохастическом случае гораздо сложнее, чем в детерминированном.

[Подробнее](https://www.mathematik.tu-darmstadt.de/media/mathematik/forschung/preprint/preprints/2354.pdf) и [ещё](https://users.aalto.fi/~ssarkka/course_s2014/handout5.pdf)

In [ ]:
class RKSolver(SDESolver):
    
    def __init__(self,b,sigma):
        super(RKSolver, self).__init__(b,sigma)
        
    def step(self, x, t, tnext,tId=None,tNextId=None,ws=None):
        """Makes Milstein step

        Args:
            x float[]: previous point (N,d)
            t float: previous time
            tnext float: next time
            tId: slice id to choose noise vector
            tNextId: slice id to choose noise vector at the next time
            ws: given noise matrix (N,d,T)
            
        Returns:
            float[]  (N,d)  step result
        """            
        #b (N,d)  x (N,d)   sigma (N,d,d)
        #raise NotImplementedError
        delta = (tnext-t)
        if(ws is None):
            dw =  #wiener process delta W   #usual regime
        else:
            dw =  #wiener process delta W   #else experimental
        #k1 = #??
        #YOUR CODE
        res = None#
        return res


### Тестируем порядок сходимости на модели GBM

Мы точно знаем решение уравнения

$$
dX_t = \mu X_t dt + \sigma X_t dW_t, \quad X_0 = x_0.
$$

Это геометрическое Броуновское движение
$$
X_t = x_0 e^{(\mu - \sigma^2/2)t + \sigma W_t}.
$$

Чтобы протестировать сильный порядок сходимости, нам понадобится по максимуму уменьшить случайность.

Чтобы оценить порядок точности, будем следовать такой схеме для каждого $\Delta$ (выбираем константный шаг по времени):
1. Задаём сетку размера $2^p+1$, чтобы можно было брать подсетки с шагом, изменяющимся в два раза.
2. Фиксируем seed (100500), семплируем $N$ траекторий GBM ($X_t$) в точках $t_0=0,....,t_{n-1}=T$.
3. Фиксируем seed (100500), семплируем $N$ траекторий Винеровского процесса в точках $t_0=0,....,t_{n-1}=T$.
4. Используя предвычисленный Винеровский процесс, считаем решения численных схем для временных сеток с шагами $1/2^k$ при $k=2,..,p$.
5. Оцениваем по Монте-Карло (потраекторно) значения ошибок сильной и слабой сходимости (слабую сейчас меряем для функции $f(x)=x^5$):
   
   $$
        \hat{\varepsilon}^{strong}_{\Delta} = \mathbb{E} \left\vert \hat{X}^{\Delta}_{t_{n-1}} - X_{T} \right\vert, \quad  \hat{\varepsilon}^{weak}_{\Delta} = \left\vert \mathbb{E}  \left(\hat{X}^{\Delta}_{t_{n-1}} - X_{T}\right) \right\vert.
   $$

Далее по полученным значениям $(\Delta_1,\hat{\varepsilon}^{strong}_{\Delta_1},\hat{\varepsilon}^{weak}_{\Delta_1}),..,(\Delta_K,\hat{\varepsilon}^{strong}_{\Delta_K},\hat{\varepsilon}^{weak}_{\Delta_K})$ строим линейную регрессии
$$
\ln\hat{\varepsilon}^{strong}_{\Delta} \sim const + \ln \Delta, \quad  \ln \hat{\varepsilon}^{weak}_{\Delta} \sim const + \ln \Delta,
$$
коэффициент перед $\Delta$ будет оценкой порядка точности.


Чтобы провести такой эксперимент, понадобится аккуратно пробросить в solver предвычисленный Винеровский процесс.

In [ ]:
class WienerDriftProcess:
    
    def __init__(self, gamma=0, sigma=1):
        """
        Args:
            gamma float[](optional): Drift coefficient (d,). Defaults to 0.
            sigma float[](optional): sigma matrix (d,d). Defaults to 1.
        """        
        self.gamma = gamma
        self.sigma = sigma
        
    def sampleTrajs(self,x0,ts,N):
        '''
        Simulates trajectories of Wiener process
        Input
        float[] x0 -- initial value (number of (d,) tensor)
        float[] ts -- times
        int N -- number of trajectories to simulate
        Returns
        float[] of shape (Ntraj,xs.shape[0],len(ts))
        '''
        try:
            x0=x0[None,:]*np.ones([N,1])
        except:
            #x0 is just a number
            x0 = np.array([x0])[None,:]*np.ones([N,1])
        xs=np.zeros(list(x0.shape)+list(ts.shape))
        #print(ts.shape, xs.shape)
        xs[...,0]=x0
        for kk in np.arange(1,len(ts)):
            noises=np.random.standard_normal(size=(N,x0.shape[1]))
            xs[...,kk]=xs[...,kk-1] + np.sum(self.sigma*np.sqrt(ts[kk]-ts[kk-1])*noises[:,None,:],axis=-1)
    
        return xs + self.gamma*ts
    
class GBMModel:
    
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        self.wien = WienerDriftProcess()
        
    def sampleTrajs(self,x0,ts):
        '''
        Simulates trajectories of GBM with parameters mu and sigma
        Input
        float[] x0 -- initial price, (N,d,)
        float[] ts -- times
        Returns
        float[] of shape (Ntraj,d,len(ts)) , trajectories
        '''
        wienProcs = self.wien.sampleTrajs(np.zeros([x0.shape[1]]),ts,x0.shape[0])
        #print(wienProcs.shape)
        if(wienProcs.shape[1]>1):
                xs = x0[...,None] * np.exp((self.mu[...,None] - self.sigma[...,None]**2/2)*(ts-ts[0]) + self.sigma[...,None]*wienProcs)
        else:
                xs = x0[...,None] * np.exp((self.mu - self.sigma**2/2)*(ts-ts[0]) + self.sigma*wienProcs)
        return xs
        

In [ ]:
seed=100500
a=0
b=1

p=10
Nts = #[construct np.array of 2**k for 2<=k<=p]  #dense time grid (N of partitions)
fullID = np.arange(2**p+0.1).astype("int32")#numbering of the nodes
tIds = #construct a lsit of np.arrays, each array giving the ids of the points in the k-th discretization (4-th is the one with 5 nodes)
ts = #construct a subdivision of a,b giving the actual node coordinates in the most dense grid

NTrajs=10000

x0=np.ones([NTrajs,1])*100
mu = 0.05
sigma=0.2
groundTruthModel = GBMModel(mu,sigma)
def bGBM(t,x):
    return mu*x
def sigmaGBM(t,x):
    try:
        #GBM sigma, (N,d,d)
    except:
        #x is a number?
        #return np.array([[[]]]) #GBM sigma (N,1,1)
def sigmaDerGBM(t,x):
    try:
        #return #GBM sigmaDerivative, (N,1,1)
    except:
        #x is a number?
        #return #GBM sigmaDerivative (N,1,1)
    

In [ ]:
def makeExperiment(deltas, solvers, groundTruth, wiens, x0, seed, tIds, ts):
    np.random.seed(seed)
    exps = {key: {"strongError":[], "weakError": []} for key in solvers.keys()}
    for key, solver in solvers.items():
        print("Solving for "+ key +"...")
        for expId in tqdm(np.arange(len(deltas))):
            timeGrid=ts[tIds[expId]]
            
            solution = solver.solve( x0, timeGrid, tIds=tIds[expId],ws=wiens, returnLast=True )
            
            exps[key]["strongError"].append( #
                                              #      compute strong error between groundTruth[...,-1]  and  solution 
                                              #  
                                            )
            exps[key]["weakError"].append( #compute weak error  
                                           #       between solution**5  and groundTruth[...,-1]**5
                                           #, axis=0) )
                                         )
        exps[key]["strongError"]=np.array(exps[key]["strongError"])
        exps[key]["weakError"]=np.array(exps[key]["weakError"])
    return exps    

def estimateReg(ers,deltas):
    #print(ers.shape,deltas.shape, (np.vstack([np.ones_like(deltas),np.log(deltas)]).T).shape)
    #given errors (N,) and deltas (N,) construct a least squares solution  (a,b) of the model 
    # log er ~ a + b log delta 
    return #??np.linalg.lstsq()
def constructFunctionFromReg(coefs):
    #prediction function, for drawing
    def fun(x):
        return np.exp(coefs[0] + coefs[1]*np.log(x))
    return fun

In [ ]:
#setting up GT 
np.random.seed(seed)
groundTruth = groundTruthModel.sampleTrajs(x0,ts)

#wienerProcesses for solvers
wienProc = WienerDriftProcess()
np.random.seed(seed)
wiens = wienProc.sampleTrajs(0,ts,NTrajs)

#init solvers
solvers = { "RK": RKSolver(bGBM, sigmaGBM), "Euler": EulerSolver(bGBM, sigmaGBM), "Milstein": MilsteinSolver(bGBM, sigmaGBM, sigmaDerGBM) }    
solverNames = list(solvers.keys())

np.random.seed(seed)

#set deltas, time step sizes
deltas = np.array([(b-a)/(Nts[expId]) for expId in np.arange(len(Nts)) ] )

#collect experiment data
experimentData = makeExperiment(deltas, solvers, groundTruth, wiens, x0, seed, tIds, ts)

#construct regressions
regs = {key: {"weakError": estimateReg(data["weakError"], deltas),
              "strongError": estimateReg(data["strongError"], deltas)} for key, data in experimentData.items()}

#construct prediction functions for regressions
regFunctions = {key: {errKey: constructFunctionFromReg(errReg) for errKey, errReg in reg.items()} for key,reg in regs.items() }

In [ ]:
regs#print regression estimates

In [ ]:
f, (ax1,ax2)  =plt.subplots(1,2, figsize=(9,4))

ax1.grid()
ax1.set_title("Weak Errors",fontsize=16)
ax1.set_xlabel('X1',fontsize=16)
ax1.set_ylabel('X2',fontsize=16)
ax1.tick_params(axis='x', labelsize=14)
ax1.tick_params(axis='y', labelsize=14)
for name in solverNames:
    ax1.plot(deltas, regFunctions[name]["weakError"](deltas))
ax1.legend(solverNames)
for name in solverNames:
    ax1.scatter(deltas, experimentData[name]["weakError"])
ax1.set_yscale("log")
ax1.set_xscale("log")

ax2.grid()
ax2.set_title("Strong Errors",fontsize=16)
ax2.set_xlabel('X1',fontsize=16)
ax2.set_ylabel('X2',fontsize=16)
ax2.tick_params(axis='x', labelsize=14)
ax2.tick_params(axis='y', labelsize=14)
for name in solverNames:
    ax2.plot(deltas, regFunctions[name]["strongError"](deltas))    
ax2.legend(solverNames)
for name in solverNames:
    ax2.scatter(deltas, experimentData[name]["strongError"])
ax2.set_yscale("log")
ax2.set_xscale("log")

Какой получился порядок сходимости?

ВАШ КОММЕНТАРИЙ

## Диффузия Ланжевена (25 баллов)


С диффузией Ланжевена мы познакомимся чуть позже, когда будем обсуждать Марковские процессы. Этот процесс задаётся как

$$
dX_t = (\nabla_x \ln p)(X_t) dt + \sigma dW_t,
$$

Этот процесс необычный тем, что он обладает свойством эргодичности. С какого бы распределения $X_0 \sim F_0$ мы не начали, $X_t$ будет сходиться с ростом времени по распределению к $X_* \sim p(\cdot)$. Благодаря этому свойству, диффузия Ланжевена стала одним из инструментов семплирования из сложных и обычно многомерных вероятностных распределений. Она даёт одно из решений проблеме семплирования в многомерном случае (где способов очень мало):
1. Почти универсальный способ (есть сложности, но вообще очень широко распространено).
2. Для любой размерности.
3. Плотность $p$ можно знать только с точностью до нормировочной константы, её часто больно считать в байесовских оценках или ЕМ-алгоритме.
4. Доступны ЦПТ(!) и ЗБЧ(!) для выборочных средних последовательных оценок, которые вообще зависимые, но со временем благодаря эргодичности как бы забывают об этом. То есть, можно просимулировать до какого-то $t=10000$ и далее уже приближённо работают ЦПТ и ЗБЧ, это так называемый разогрев (burn-in period).

Но есть и минусы:
1. Можно долго ждать сходимости, если не повезёт с начальным распределением или с распределением $p$.
2. Нужно экспериментировать с параметрами численного метода, чтобы получить приемлемый уровень сходимости и не ждать долго.
3. Чтобы обойти все моды, нужно прибегать к дополнительным трюкам.


Такой метод называется Unadjusted Langevin Algorithm или коротко ULA.

### Простой пример (10 баллов)

Давайте посмотрим, что будет с семплами и с траекториями, если взять (с точностью до нормировки)

$$
p(x,y)= exp\left(-\frac{0.8x^2 + (y-x^{2/3})^2}{4}\right).
$$

Используя численные методы, просимулируйте 500 траекторий до достаточно позднего момента (например, $1000$), задав начальное условие из распределения $N(0,0.2I)$. 

Нарисуйте ниже на разных графиках с помощью scatter положение траекторий на плоскости в начальный момент и в моменты (примерно) $250,500,750,1000$.

На отдельном графике с помощью plot нарисуйте одну траекторию.

Поисследуйте с помощью гистограмм, когда примерно заканчивается период прогрева?

In [ ]:
a=0
b=10000
Nt=20000
h=(b-a)/Nt
ts = np.arange(a,b+h/2,h)

sigma=0.5

Ntraj=100
x0 = np.random.uniform(-1,20,size=(Ntraj,2))#np.ones([Ntraj,2])

def bLan(t,x):
    #x (N,d)
    #return #YOUR CODE
    #return shape (N,d)
def sigmaLan(t,x):
    #x (N,d)
    #return #YOUR CODE
    #return shape (N,d,d)

solver = EulerSolver(bLan,sigmaLan)

In [ ]:
apprTrajs = solver.solve(x0,ts)
print(apprTrajs.shape)

In [ ]:
#plots


In [ ]:
#plots

ВАШИ ПОЯСНЕНИЯ

In [ ]:
#ВАШ КОД

ВАШИ ПОЯСНЕНИЯ

## Красивый художественный эффект (15 баллов)

Давайте попробуем сделать что-то красивое. Диффузия Ланжевена по своей конструкции случайно блуждает по пространству, но как бы тяготеет к модам распределения. Случайные блуждания можно использовать для получения интересных художественных эффектов.

Давайте считаем фотографию и обратим своё внимание на канал яркости. Если изображение более светлое, то на контрасте будут скорее самые тёмные места и наоборот. Нам понадобится openCV (импортирован в начале как cv2).

In [ ]:
def read_image(filename):
        """Reads an image and returns a RGB image. If the imported image has an alpha channel, this is
        blended as if there were a white canvas underneath in order to give
        a RGB image

        Args:
            filename (_type_): _description_

        Returns:
            _type_: _description_
        """        
        
        # Import in the image
        img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

        # Handle transparency
        if (img.shape[2] == 4):
            img_rgb = img[:, :, :3]
            img_alpha = img[:, :, 3]

            alpha = img_alpha[:, :, np.newaxis].astype(np.float32) / 255.0
            alpha = np.concatenate((alpha, alpha, alpha), axis=2)

            img = img_rgb.astype(np.float32)*alpha + \
                np.ones_like(img_rgb)*255*(1-alpha)
            img = img.astype(np.uint8)

        return img

In [ ]:
inputFilename="./HSE.jpg"
img = read_image(inputFilename)
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)#since opencv permutes channels to BGR

In [ ]:
f, ax = plt.subplots(figsize=(9,5))

ax.set_title("HSE Atrium")
ax.imshow(imgRGB)

Чтобы получить яркость, нужно усреднить по цветовому каналу и отнормировать на 255(максимальное значение цвета).

In [ ]:
brightnessDensity = 1-np.mean(imgRGB,axis=-1)/255#darkness map
brightnessDensity = np.mean(imgRGB,axis=-1)/255#brightness map

In [ ]:
f, (ax1,ax2) = plt.subplots(1,2,figsize=(18,5))

ax1.set_title("HSE Atrium")
ax1.imshow(imgRGB)

ax2.set_title("HSE Atrium, Darkness")
im2=ax2.imshow(brightnessDensity,cmap='hot')
plt.colorbar(im2, ax=ax2)

Мы попробуем "нарисовать" фотографию используя диффузию Ланжевена.

Процесс будет путешествовать по картинке, а траектория будет рисоваться на пустом белом холсте, отмечая один шаг процесса отрезком:
1. В начальной и конечной точке рисуется круг цвета ближайшего к точке пикселя
2. Две точки соединяются линией с цветным градиентом от одной точке к другой (режим $\texttt{blend=True}$) или цвета стартовой точки (режим $\texttt{blend=False}$).
 
Для управления движением диффузии зададим плотность в виде гауссовской смеси в некоторых опорных точках $x_i$ (anchors). Одна из идей -- случайно выбрать $N$ точек и задать гауссовскую смесь с центрами в них, фиксированной дисперсией $\sigma_G^2$ и весами смеси $\alpha_1,..,\alpha_N$, равными яркости (или темноте) пикселя, где находится центр (с учётом последуещй нормировки к единице). Получится плотность

$$
p(x) = \sum_{i=1}^N \alpha_i N(x; x_i, ~ \sigma_G^2), \quad \sum_{i=1}^N \alpha_i = 1,
$$
где $N(x; x_i, ~ \sigma_G^2)$ -- значение гауссовской плотности распределения с параметрами $x_i,\sigma_G^2$. Так более вероятными регионами посещения блуждания будут те, где плотность имеет локальный максимум (то есть, в опорных точках), а размер максимума соотносится с яркостью (или наоборот темнотой).

Далее из случайной точки запускаем диффузию на какое-то время и потом сохраняем картинку в файл.

#### Плотность и задание диффузии

Начнём с того, чтобы задать плотность и градиент логарифма для диффузии.

Напишите, чему равен градиент логарифма плотности гауссовской смеси выше.

ВАШ КОММЕНТАРИЙ

In [ ]:
def constructSDE(imgRGB,nAnchors=100, sigma=100,sigmaNoise=100, darkness=True):
        """Returns SDE parameters for artist

        Args:
            imgRGB (float[]): RGB image in shape (R,G,B)
            nAnchors (int, optional): number of anchors to use for gaussian mixture. Defaults to 100.
            sigma (int, optional): varianvce parameter sigma_G for gaussian mixture. Defaults to 100.
            sigmaNoise (int, optional): noise variance for SDE. Defaults to 100.
            darkness bool: whether to use darkness map, otherwise brightness map

        Returns:
            FuncHandler density, FuncHandler densGradLog, FuncHandler noiseSig, tuple (float[] anchors, float[] weights)
        """    

        if(darkness):
            brightnessDensity = 1 - np.mean(imgRGB,axis=-1)/255
        else:
            brightnessDensity = 1 - np.mean(imgRGB,axis=-1)/255
        weights = np.reshape(brightnessDensity,-1)
        points = np.vstack([np.kron(np.arange(imgRGB.shape[0]),np.ones([imgRGB.shape[1]])),np.kron(np.ones([imgRGB.shape[0]]),np.arange(imgRGB.shape[1])) ]).T
        
        randomIds = np.random.choice(np.arange(weights.shape[0]), p=weights/np.sum(weights),size=nAnchors, replace=False)
        anchorWeights = weights[randomIds]
        anchorWeights = anchorWeights/np.sum(anchorWeights)
       
        anchors = points[None,randomIds,:]
        sigma=100
        def densGradLog(t,x):
            #x (N,2)   points (Np,2)
            #result (N,2)
            if (x.ndim==1):
                x = x[None,:]
            res = #YOUR CODE
            return  res

        def noiseSig(t,x):
            #x (N,2) 
            #return #YOUR CODE
            #return shape (N,2,2)
        
        def dens(x):
            #x (N,2)   points (Np,2)
            #return 0 
            if (x.ndim==1):
                x = x[None,:]
            #res = ???? # YOUR CODE
            #return  res
            #return shape (N,2,2)
        
        return dens, densGradLog, noiseSig, (anchors[0,...],anchorWeights)

Чтобы проверить, как задаётся плотность, давайте попробуем отобразить её, центры смеси и сравнить с картинкой.

In [ ]:
nAnchors = 1000
sigmaNoise = 50
sigma=100

#init SDE
density, bFun , sigFun, (points,weights) = constructSDE(img,nAnchors=nAnchors,sigma=sigma, sigmaNoise=sigmaNoise)

In [ ]:
f, (ax1,ax2) = plt.subplots(1,2,figsize=(12,4))

ax1.set_title("Anchors")
ax1.scatter(points[:,1],-points[:,0])
#print(weights)

ylim, xlim = brightnessDensity.shape
xs=np.arange(0,xlim,5)#tune these for better resolution in heatmap
ys=np.arange(0,ylim,5)#
xx,yy = np.meshgrid(xs,ys)
pps = np.vstack([np.kron(ys,np.ones_like(xs)),np.kron(np.ones_like(ys),xs)]).T
denss = density(pps).squeeze().reshape((ys.shape[0],xs.shape[0]))

ax2.set_title("Distribution of Mixture")
im2=ax2.imshow(denss,cmap='hot')
plt.colorbar(im2, ax=ax2)

In [ ]:
f, (ax1,ax2) = plt.subplots(1,2,figsize=(18,5))

ax1.set_title("HSE Atrium")
ax1.imshow(imgRGB)

ax2.set_title("HSE Atrium, Darkness")
im2=ax2.imshow(brightnessDensity,cmap='hot')
plt.colorbar(im2, ax=ax2)

### Генерация траекторий

Координату процесса $X_t$ будет задавать диффузия Ланжевена

$$
dX_t = (\nabla_x \ln p)(X_t) dt + \sigma dW_t,
$$

где в качестве плотности мы возьмём плотность гауссовской смеси.

Разрешим в качестве параметра задавать количество траекторий, то есть, чтобы стартовая точка ресемплилась и оттуда запускалась новая траектория, для этого введём параметр $N_{trajs}$.

Также нужно будет предусмотреть механизм, держащий траекторию в пределах фото, то есть, чтобы она не выходила за рамки картинки. В качестве идеи, просто добавляем гауссовский сдвиг с нулевым матожиданием и дисперсией, пытаясь вернуть траекторию в область фото.

Траекторию будем складывать в список из точек $(x,y)$. Траекторий много, поэтому будем их хранить как список списков.

Наконец, нужно реализовать рисование сгенерированных траекторий. Это уже во многом сделано, чтобы сэкономить время на выяснение деталей в openCV, но некоторые технические моменты придётся доделать.

Всё это реализуем в рамках класса ULArtist. 

In [ ]:
class ULArtist:
    
    def __init__(self, img, solver):
        '''
            float[] img: opencv img (height,width,3), RGB
            solver Obj: SDESolver object
        '''
        self.img = img
        self.solver=solver
        self.imgDims = self.img.shape[:2]


    def genNewPixelULA(self,loc,t, delta):
        """Generates new pixel starting from loc at time t and with step delta

        Args:
            loc float[]: start (2,)
            t float: time
            delta float: time step

        Returns:
            float[] newPixel
        """        
        ylim, xlim = self.imgDims
        
        #use solver to generate new pixel
        #newPixel = ??? shape (2,)
        
        #clip it
        newPixel = np.array([np.clip(newPixel[0],0,ylim),np.clip(newPixel[1],0,xlim)])
        return newPixel
    
    
    def ulaTransformer(self, initialPoint, nIterations,delta):
        """Uses ULA algorithm to design a drawing path

        Args:
            initialPoint float[][]: point to start the walk
            nIterations int: number of steps in the trajectory
            solver Obj: SDESolver object implementing method step

        Returns:
            list of float[], points of the random walk
        """    

        # Preallocation and obtaining useful information from the input args
        sampleList = [None]*(nIterations + 1)
        sampleList[0] = initialPoint
        
        # Generate samples from brightnessDensity in terms of pixel locations
        for i in tqdm(range(nIterations), desc='Sampling'):
            newPixel = self.genNewPixelULA(sampleList[i], i, delta)
            while(np.linalg.norm(newPixel-sampleList[i])<=1e-8):
                #if converged on border, then punch the particle
                start = sampleList[i] + np.random.normal(size=(2,))*10
                start = np.array([np.clip(start[0],0,self.imgDims[0]),np.clip(start[1],0,self.imgDims[1])])
                newPixel = self.genNewPixelULA(start, i, delta)
            #print("NP",newPixel)
            sampleList[i+1] = newPixel

        return sampleList


    def sampleInitialPoint(self):
        '''
        Gives a point to begin sampling from.
        '''
        ylim, xlim = self.imgDims
        initialPoint = np.random.multivariate_normal(mean=np.array([ylim/2,xlim/2]),cov=np.eye(2)*100)
        initialPoint = np.clip(initialPoint, 0, [xlim, ylim])
        return initialPoint
    

    def drawTraj(self, imgNew, sampleList, blend, pointRadius, lineWidth):
        """Draws a trajectory onto canvas given by imgNew

        Args:
            imgNew float[]: image to draw on (openCV)
            sampleList list(float[]): list of visited points
            blend (_type_): _description_
            pointRadius (_type_): _description_
            lineWidth (_type_): _description_
        """        
        if blend:#if so, add a color gradient to the line
            imgX = sampleList[0][0]
            imgY = sampleList[0][1]
            imgNew = cv2.circle(imgNew, (imgX,imgY) ,
                                        radius=5,
                                        color=(255,0,0),
                                        thickness=-1)
            for i in tqdm(np.arange(len(sampleList)-1), desc='Rendering Blended'):
                # Grab colors of start and end pixels
                
                imgX = sampleList[i][0]
                imgY = sampleList[i][1]
                
                start_color = img[imgX, imgY, :]
                end_color = img[imgX, imgY, :]

                # Render a dot
                imgNew = cv2.circle(imgNew, sampleList[i],
                                        radius=pointRadius,
                                        color=tuple([int(x) for x in start_color]),
                                        thickness=-1)

                # Prepare a line
                new_line = list(zip(*skline(*sampleList[i], *sampleList[i+1])))#color gradient
                blender = np.linspace(0, 1, len(new_line))

                start_color = np.array(start_color)
                end_color = np.array(end_color)
                color_mix = start_color[:, np.newaxis] * \
                    (1-blender) + end_color[:, np.newaxis]*blender
                color_mix = color_mix.astype(np.uint8).transpose()

                # render the line
                for j in range(len(new_line)):
                    imgNew[new_line[j][::-1]] = color_mix[j]
        else:

            for i in tqdm(np.arange(len(sampleList)-1), desc='Rendering'):
                # Grab color of starting pixel
                imgX = sampleList[i][0]
                imgY = sampleList[i][1]
                img_color = img[imgX, imgY, :]
                img_color = tuple([int(x) for x in img_color])

                # Render dots and lines to canvas
                imgNew = cv2.circle(imgNew, sampleList[i],
                                        radius=pointRadius,
                                        color=img_color,
                                        thickness=-1)
                imgNew = cv2.line(imgNew, sampleList[i], sampleList[i+1],
                                    img_color, lineWidth)
                
                

    def drawImage(self, outputFilename, nIterations, 
                  delta=0.01,
                  pointRadius=1,
                  lineWidth=1,
                  blend=False,
                  Ntrajs=10
                  ):
        """Draws an mcmcfied picture onto new canvas

        Args:
            outputFilename str: Path to save output image
            nIterations int: number of steps in one trajectory
            delta float: time step for solver to use. Defaults to 0.01.
            pointRadius (int, optional): size of the point on canvas (openCV has minimal size 1). Defaults to 1.
            lineWidth (int, optional): width of the line on canvas (openCV has minimal width 1). Defaults to 1.
            blend (bool, optional): whether to use color gradient on the lines. Defaults to False.
            nLists (int, optional): number of trajectories. Defaults to 10.

        Raises:
            Exception: _description_
        """        
        
        #canvas init
        imgNew = 255*np.ones((self.imgDims[1], self.imgDims[0], 3), np.uint8)
        for j in np.arange(Ntrajs):
            print(f"{j+1} trajs of {Ntrajs} processing....")
            
            #sample starting point
            initialPoint = self.sampleInitialPoint()
            
            #construct trajectory
            sampleList = self.ulaTransformer(initialPoint, nIterations, delta=delta)
            
            #round it (opencv requires int)
            sampleList = [ tuple(np.ceil(sampleList[i]).astype("int32").reshape(1,-1)[0]-1) for i in np.arange(len(sampleList))]
            
            #draw the trajectory onto the canvas
            self.drawTraj(imgNew, sampleList, blend, pointRadius, lineWidth)    
            
        print("DONE")

        # Export image
        imgNew = np.swapaxes(imgNew[:, :, :], 0, 1)
        if not cv2.imwrite(outputFilename, imgNew):
            raise Exception("Could not write image.")
        else:
            print('Image saved to '+outputFilename)

Давайте запустим! Загрузите своё любимое фото (лучше разрешением побольше, хотя бы 1000х1500+) и нарисуйте обработанное.

*Чисто в теории возможно, что алгоритм зависнет, очень редко такое возможно, в этом случае помогает его перезапуск.

In [ ]:
inputFilename=#YOUR FAVORITE PHOTO
outputFilename="./MyPhotoMCMCfied.jpg"
nIterations=35000
Ntrajs = 1
img = read_image(inputFilename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)#since opencv permutes channels to BGR

#brightnessDensity = 1-np.mean(imgRGB,axis=-1)/255#darkness map
brightnessDensity = np.mean(img,axis=-1)/255#brightness map

delta = 0.1

nAnchors = 1000
sigma=200
sigmaNoise=40
blend=True
density, bFun , sigFun,  (points,weights) = constructSDE(img,nAnchors=nAnchors,sigma=sigma, sigmaNoise=sigmaNoise)
solver = EulerSolver(bFun, sigFun) #your favorite solver
artist = ULArtist(img,solver)

In [ ]:
artist.drawImage(outputFilename,nIterations, delta=delta, Ntrajs=Ntrajs,blend=blend)

И давайте нарисуем.

In [ ]:
img2 = read_image(outputFilename)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)#since opencv permutes channels to BGR

In [ ]:
f, ax = plt.subplots(figsize=(9,5))

ax.set_title("Original Picture")
ax.imshow(img)

In [ ]:
f, ax = plt.subplots(figsize=(9,5))

ax.set_title("Processed Picture")
ax.imshow(imgRGB)

### Эксперименты

Нарисуйте фото с небольшим параметром $\texttt{nIterations}=20000$ и с большим $\texttt{nIterations}=500000$. Опишите и дайте объяснение эффекту.

In [ ]:
#YOUR CODE

ВАШ КОММЕНТАРИЙ

Теперь попробуйте поменять параметр шага $\texttt{delta}$ и дисперсию диффузии $\texttt{sigmaNoise}$. Опишите и поясните эффект.

In [ ]:
#YOUR CODE

ВАШ КОММЕНТАРИЙ

В чём отличие от обычного Винеровского процесса? Попробуйте используя тот же пайплайн нарисовать картину чисто двумерным Винеровским процессом (для этого нужно в качестве функции $b$ в солвер задать функцию тождественного 0).

In [ ]:
#YOUR CODE

ВАШ КОММЕНТАРИЙ